In [1]:
import torch
from torch import nn

In [2]:
input = torch.randn(24, 2915)

In [3]:
seq_len = input[:, -1].to(torch.long)
seq_len

tensor([ 0,  0,  0,  1,  0, -1,  1,  0, -1,  0,  0, -1,  0,  0,  0,  2,  1,  0,
         1,  0,  0,  0,  1, -1])

In [52]:
raw_in = input[:, : 6 * 240].reshape(-1, 30, 6).transpose(1, 2)
raw_in.shape

torch.Size([192, 6, 30])

In [5]:
dnn_in = input[:, 6 * 240 : -1].reshape(24, -1, 2)
dnn_in.shape

torch.Size([24, 737, 2])

In [48]:
raw_in.shape

torch.Size([192, 6, 30])

In [79]:
cnn = nn.Sequential(nn.Conv1d(6, 3, 3), nn.ReLU(),)

In [77]:
cnn(raw_in).view(24, 8, -1).shape

torch.Size([24, 8, 168])

In [80]:
print(cnn)

Sequential(
  (0): Conv1d(6, 3, kernel_size=(3,), stride=(1,))
  (1): ReLU()
)


In [117]:
cnn1 = nn.Sequential(nn.Conv1d(6, 3, 1), nn.ReLU(), nn.Conv1d(3, 3, 3), nn.ReLU())

In [119]:
cnn1(raw_in).view(24, 8, -1).shape

torch.Size([24, 8, 84])

In [113]:
cnn = nn.Sequential(nn.Conv1d(6, 3, 1), nn.ReLU(), nn.Conv1d(6, 3, 1), nn.ReLU(), )

In [106]:
maxPool = nn.MaxPool1d(kernel_size=1)

In [107]:
maxPool(cnn_out).shape

torch.Size([24, 8, 84])

In [108]:
drop = nn.Dropout(p=0.01)

In [109]:
drop(cnn_out).shape

torch.Size([24, 8, 84])

In [13]:
dnn_in = input[:, 6 * 240 : -1].reshape(24, -1, 2)
dnn_in.shape

torch.Size([24, 737, 2])

In [84]:
raw_fc = nn.Sequential(nn.Linear(84, 64), nn.ReLU(), nn.Linear(64, 64), nn.ReLU(),)

In [97]:
rnn = nn.LSTM(64, 64, batch_first=True, num_layers=2)

In [89]:
b = raw_fc(cnn_out)

In [98]:
c = rnn(b)[0][:, -1, :]

In [99]:
c.shape

torch.Size([24, 64])

In [14]:
dnn = nn.Sequential(nn.Linear(2, 64), nn.ReLU(), nn.Linear(64, 64), nn.ReLU(),)

In [21]:
dnn(dnn_in).shape

torch.Size([24, 737, 64])

In [16]:
rnn2 = nn.GRU(64, 64, batch_first=True)

In [31]:
a = rnn2(dnn(dnn_in))[0]

In [32]:
a.shape

torch.Size([24, 737, 64])

In [33]:
a = a[torch.arange(a.size(0)), seq_len]

In [34]:
a.shape

torch.Size([24, 64])

In [92]:
torch.cat((c, a), dim=-1).shape

torch.Size([24, 128])